In [80]:
%load_ext autoreload
%autoreload 2

In [89]:
from os.path import abspath, curdir, join, exists
import gffutils
import sqlite3

In [82]:
data_dir = abspath(join(curdir, "..", "data"))
public_dir = abspath(join(curdir, "..", "frontend", "public"))

GCF_000002765.6.gff from the assembly of Plasmodium falciparum 3D7: https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000002765.6/

In [113]:
recreate = False
if not exists(join(public_dir, "GCF_000002765.6.db")) or recreate:
    gff_db = gffutils.create_db(
        join(data_dir, "GCF_000002765.6", "genomic.gff"),
        join(public_dir, "GCF_000002765.6.db"),
        merge_strategy="merge",
        force=True,
    )
else:
    gff_db = gffutils.FeatureDB(join(public_dir, "GCF_000002765.6.db"))

In [109]:
# gff_db = sqlite3.connect(join(public_dir, "GCF_000002765.6.db"))
gff_db.conn.execute(
    "create table if not exists sequences (features_id text primary key, seq text, foreign key(features_id) references features(id))"
)

In [115]:
fna = join(data_dir, "GCF_000002765.6", "GCF_000002765.6_GCA_000002765_genomic.fna")
for feature in gff_db.all_features():
    try:
        seq = feature.sequence(fna)
        gff_db.conn.execute("insert into sequences values (?, ?)", (feature.id, seq))
        gff_db.conn.commit()
    except Exception as e:
        print(e)
        continue

UNIQUE constraint failed: sequences.features_id
UNIQUE constraint failed: sequences.features_id
